In [9]:
import pandas as pd
import os
import re
import shutil

# 예결산서 sample file

## 파일 이름 변경

### dry run

In [1]:
import os
import re
import unicodedata

def parse_filename(filename: str):
    # 1. (자세히보기) 제거
    filename = re.sub(r'\(자세히보기\)', '', filename)

    # 2. (1) (2) 추출 (연도 결정)
    year = "2022"  # 기본
    match_year = re.search(r'\((\d)\)', filename)
    if match_year:
        if match_year.group(1) == "1":
            year = "2023"
        elif match_year.group(1) == "2":
            year = "2024"
        filename = re.sub(r'\(\d\)', '', filename)

    # 3. 복사본 처리: 공백 없애고 숫자 추출
    filename = re.sub(r'복사본\s+', '복사본', filename)  # 복사본 공백 제거
    copy_number = 1
    match_copy = re.search(r'복사본(\d*)', filename)
    if match_copy:
        if match_copy.group(1):
            copy_number = int(match_copy.group(1)) + 1
        else:
            copy_number = 2
        filename = re.sub(r'복사본\d*', '', filename)

    # 4. 국_공립 → 국공립
    filename = filename.replace('국_공립', '국공립')

    # 5. 중간에 끼어있는 .xls, .xlsx 제거
    filename = re.sub(r'\.xls[x]?', '', filename)

    # 6. 공백 정리
    filename = re.sub(r'\s+', ' ', filename).strip()

    return filename, year, copy_number

def dry_run(base_dir="data"):
    for root, _, files in os.walk(base_dir):
        norm_root = unicodedata.normalize('NFC', root)

        # 학교급 판단
        if "초등" in norm_root:
            school_level = "초등"
        elif "중등" in norm_root:
            school_level = "중등"
        elif "고등" in norm_root:
            school_level = "고등"
        else:
            school_level = None

        for file in files:
            if file.endswith(".xls") or file.endswith(".xlsx"):
                old_path = os.path.join(root, file)

                norm_file = unicodedata.normalize('NFC', file)

                # 이름 파싱
                base_name, year, copy_number = parse_filename(norm_file)

                level_tag = school_level if school_level else "기타"

                # 최종 새 파일명
                new_filename = f"{year}_{base_name}_{level_tag}_{copy_number}.xls"

                print(f"[DRY-RUN] {old_path} ➔ {new_filename}")

# 실행
dry_run()

[DRY-RUN] data/광주/초등/국_공립 학교회계 예・결산서(자세히보기) (1).xls ➔ 2023_국공립 학교회계 예・결산서_초등_1.xls
[DRY-RUN] data/광주/초등/국_공립 학교회계 예・결산서(자세히보기).xls ➔ 2022_국공립 학교회계 예・결산서_초등_1.xls
[DRY-RUN] data/광주/초등/국_공립 학교회계 예・결산서(자세히보기) (2).xls ➔ 2024_국공립 학교회계 예・결산서_초등_1.xls
[DRY-RUN] data/광주/초등/2022/2022_국공립 학교회계 예・결산서_초등_1.xls ➔ 2022_2022_국공립 학교회계 예・결산서_초등_1_초등_1.xls
[DRY-RUN] data/광주/초등/2024/2024_국공립 학교회계 예・결산서_초등_1.xls ➔ 2022_2024_국공립 학교회계 예・결산서_초등_1_초등_1.xls
[DRY-RUN] data/광주/초등/2023/2023_국공립 학교회계 예・결산서_초등_1.xls ➔ 2022_2023_국공립 학교회계 예・결산서_초등_1_초등_1.xls
[DRY-RUN] data/세종/초등/국_공립 학교회계 예・결산서(자세히보기) (1).xls ➔ 2023_국공립 학교회계 예・결산서_초등_1.xls
[DRY-RUN] data/세종/초등/국_공립 학교회계 예・결산서(자세히보기).xls ➔ 2022_국공립 학교회계 예・결산서_초등_1.xls
[DRY-RUN] data/세종/초등/국_공립 학교회계 예・결산서(자세히보기) (2).xls ➔ 2024_국공립 학교회계 예・결산서_초등_1.xls
[DRY-RUN] data/제주/초등/국_공립 학교회계 예・결산서(자세히보기) (1).xls ➔ 2023_국공립 학교회계 예・결산서_초등_1.xls
[DRY-RUN] data/제주/초등/국_공립 학교회계 예・결산서(자세히보기).xls ➔ 2022_국공립 학교회계 예・결산서_초등_1.xls
[DRY-RUN] data/제주/초등/국_공립 학교회계 예・결산서(자세히보기) (2).xls

### 실제 실행

In [40]:
import os
import re
import shutil
import unicodedata

def parse_filename(filename: str):
    filename = re.sub(r'\(자세히보기\)', '', filename)
    year = "2022"
    match_year = re.search(r'\((\d)\)', filename)
    if match_year:
        if match_year.group(1) == "1":
            year = "2023"
        elif match_year.group(1) == "2":
            year = "2024"
        filename = re.sub(r'\(\d\)', '', filename)
    filename = re.sub(r'복사본\s+', '복사본', filename)
    copy_number = 1
    match_copy = re.search(r'복사본(\d*)', filename)
    if match_copy:
        if match_copy.group(1):
            copy_number = int(match_copy.group(1)) + 1
        else:
            copy_number = 2
        filename = re.sub(r'복사본\d*', '', filename)
    filename = filename.replace('국_공립', '국공립')
    filename = re.sub(r'\.xls[x]?', '', filename)
    filename = re.sub(r'\s+', ' ', filename).strip()
    return filename, year, copy_number

def copy_and_rename(base_dir="data"):
    for region in os.listdir(base_dir):
        region_path = os.path.join(base_dir, region)
        if not os.path.isdir(region_path):
            continue
        for school_level in ["초등", "중등", "고등"]:
            level_path = os.path.join(region_path, school_level)
            if not os.path.isdir(level_path):
                continue

            # level_path 바로 아래 파일만 대상으로
            for file in os.listdir(level_path):
                file_path = os.path.join(level_path, file)
                if os.path.isfile(file_path) and (file.endswith(".xls") or file.endswith(".xlsx")):
                    norm_file = unicodedata.normalize('NFC', file)
                    base_name, year, copy_number = parse_filename(norm_file)
                    level_tag = school_level

                    new_filename = f"{year}_{base_name}_{level_tag}_{copy_number}.xls"
                    target_folder = os.path.join(level_path, year)
                    os.makedirs(target_folder, exist_ok=True)

                    new_path = os.path.join(target_folder, new_filename)

                    print(f"[COPY] {file_path} ➔ {new_path}")
                    shutil.copy2(file_path, new_path)

In [41]:
copy_and_rename()

[COPY] data/광주/초등/국_공립 학교회계 예・결산서(자세히보기) (1).xls ➔ data/광주/초등/2023/2023_국공립 학교회계 예・결산서_초등_1.xls
[COPY] data/광주/초등/국_공립 학교회계 예・결산서(자세히보기).xls ➔ data/광주/초등/2022/2022_국공립 학교회계 예・결산서_초등_1.xls
[COPY] data/광주/초등/국_공립 학교회계 예・결산서(자세히보기) (2).xls ➔ data/광주/초등/2024/2024_국공립 학교회계 예・결산서_초등_1.xls
[COPY] data/제주/초등/국_공립 학교회계 예・결산서(자세히보기) (1).xls ➔ data/제주/초등/2023/2023_국공립 학교회계 예・결산서_초등_1.xls
[COPY] data/제주/초등/국_공립 학교회계 예・결산서(자세히보기).xls ➔ data/제주/초등/2022/2022_국공립 학교회계 예・결산서_초등_1.xls
[COPY] data/제주/초등/국_공립 학교회계 예・결산서(자세히보기) (2).xls ➔ data/제주/초등/2024/2024_국공립 학교회계 예・결산서_초등_1.xls
[COPY] data/경북/초등/국_공립 학교회계 예・결산서(자세히보기) 복사본 2.xls ➔ data/경북/초등/2022/2022_국공립 학교회계 예・결산서_초등_3.xls
[COPY] data/경북/초등/국_공립 학교회계 예・결산서(자세히보기) (1).xls ➔ data/경북/초등/2023/2023_국공립 학교회계 예・결산서_초등_1.xls
[COPY] data/경북/초등/국_공립 학교회계 예・결산서(자세히보기) (1) 복사본 2.xls ➔ data/경북/초등/2023/2023_국공립 학교회계 예・결산서_초등_3.xls
[COPY]

## 파일에서 추출

In [14]:
import pandas as pd
import os
from bs4 import BeautifulSoup

def parse_html_file(file_path: str, folder_year: int):
    """
    HTML 파일에서 세입예산, 세출예산, 세입결산, 세출결산 테이블과
    학교명, 학년도 정보를 추출합니다.
    """
    # 파일명 앞에 숫자(년도)가 없는 경우 바로 스킵
    filename = file_path.split("/")[-1]
    if not filename[:4].isdigit():
        return None

    with open(file_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

    # 모든 boxct div를 찾는다
    boxcts = soup.find_all("div", class_="boxct")
    
    # boxct가 4개보다 적으면 잘못된 파일 → 스킵
    if len(boxcts) < 4:
        print(f"⚠️ div.boxct 부족: {file_path}")
        return None

    # 각 테이블 추출
    seiip_table = boxcts[0].find("table")
    seichul_table = boxcts[1].find("table")
    seiip_result_table = boxcts[2].find("table")
    seichul_result_table = boxcts[3].find("table")

    # 각 학년도 추출 (세입예산에서 가져오면 충분)
    extra_info = boxcts[0].find("span", class_="extra_info")
    if extra_info is None:
        print(f"❌ 학년도 정보 없음: {file_path}")
        return None
    
    extra_text = extra_info.get_text()
    import re
    year_match = re.search(r"(\d{4})학년도", extra_text)
    if not year_match:
        print(f"❌ 학년도 추출 실패: {file_path}")
        return None

    학년도 = int(year_match.group(1))

    # 학년도+1과 폴더명 기준 year 비교
    if 학년도 + 1 != folder_year:
        print(f"❌ 학년도 +1 불일치: 파일 {filename}, 학년도 {학년도}, 폴더년도 {folder_year}")
        return None

    # 학교 이름 추출
    manager_info = soup.find("div", class_="manager_info")
    if manager_info is None:
        print(f"❌ 학교정보 div 없음: {file_path}")
        return None

    school_text = manager_info.get_text()
    school_name_match = re.search(r"학교\s*:\s*(.*?)\s*\|", school_text)
    if not school_name_match:
        print(f"❌ 학교명 추출 실패: {file_path}")
        return None
    school_name = school_name_match.group(1)

    # 결과 반환
    return {
        "file_path": file_path,
        "school_name": school_name,
        "학년도": 학년도,
        "tables": {
            "세입예산": str(seiip_table),
            "세출예산": str(seichul_table),
            "세입결산": str(seiip_result_table),
            "세출결산": str(seichul_result_table),
        }
    }

def save_parsed_tables(parsed_result: dict, output_dir: str):
    """
    parse_html_file() 결과를 받아 네 개 테이블을 CSV로 저장
    """
    school_name = parsed_result["school_name"]
    file_year = parsed_result["학년도"] + 1  # 저장파일은 학년도+1 기준
    tables = parsed_result["tables"]

    # output_dir/년도/폴더를 만든다
    target_folder = os.path.join(output_dir, str(file_year))
    os.makedirs(target_folder, exist_ok=True)

    for table_name, table_html in tables.items():
        # HTML 테이블을 DataFrame으로 변환
        df = pd.read_html(table_html)[0]

        # 저장할 파일명
        safe_school_name = school_name.replace(" ", "_")  # 파일명 안전하게
        filename = f"{file_year}_{safe_school_name}_{table_name}.csv"
        output_path = os.path.join(target_folder, filename)

        # 저장
        df.to_csv(output_path, index=False, encoding="utf-8-sig")
        print(f"✅ 저장 완료: {output_path}")

In [11]:
# file_path = "data/경남/고등/2022/2022_국공립 학교회계 예・결산서_고등_1.xls"
# folder_year = 2022
# output_dir = "Database/sample_schoolinfo"

# # Step 1. 파일 파싱
# parsed_result = parse_html_file(file_path, folder_year)

# # Step 2. 성공했으면 저장
# if parsed_result:
#     save_parsed_tables(parsed_result, output_dir)
# else:
#     print("⚠️ 파싱 실패로 저장 스킵")

In [15]:
import pandas as pd
import os
import re
from bs4 import BeautifulSoup

def parse_html_file(file_path: str, folder_year: int):
    filename = os.path.basename(file_path)
    if not filename[:4].isdigit():
        return None

    with open(file_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

    boxcts = soup.find_all("div", class_="boxct")
    if len(boxcts) < 4:
        print(f"⚠️ div.boxct 부족: {file_path}")
        return None

    seiip_table = boxcts[0].find("table")
    seichul_table = boxcts[1].find("table")
    seiip_result_table = boxcts[2].find("table")
    seichul_result_table = boxcts[3].find("table")

    extra_info = boxcts[0].find("span", class_="extra_info")
    if extra_info is None:
        print(f"❌ 학년도 정보 없음: {file_path}")
        return None
    
    extra_text = extra_info.get_text()
    year_match = re.search(r"(\d{4})학년도", extra_text)
    if not year_match:
        print(f"❌ 학년도 추출 실패: {file_path}")
        return None
    학년도 = int(year_match.group(1))

    if 학년도 + 1 != folder_year:
        print(f"❌ 학년도 +1 불일치: 파일 {filename}, 학년도 {학년도}, 폴더년도 {folder_year}")
        return None

    manager_info = soup.find("div", class_="manager_info")
    if manager_info is None:
        print(f"❌ 학교정보 div 없음: {file_path}")
        return None

    school_text = manager_info.get_text()
    school_name_match = re.search(r"학교\s*:\s*(.*?)\s*\|", school_text)
    if not school_name_match:
        print(f"❌ 학교명 추출 실패: {file_path}")
        return None
    school_name = school_name_match.group(1)

    return {
        "file_path": file_path,
        "school_name": school_name,
        "학년도": 학년도,
        "tables": {
            "세입예산": str(seiip_table),
            "세출예산": str(seichul_table),
            "세입결산": str(seiip_result_table),
            "세출결산": str(seichul_result_table),
        }
    }

def save_parsed_tables(parsed_result: dict, input_path: str, output_dir: str):
    school_name = parsed_result["school_name"]
    file_year = parsed_result["학년도"] + 1
    tables = parsed_result["tables"]

    # input_path로부터 시도명/학교급명 추출
    parts = input_path.split(os.sep)
    try:
        idx = parts.index("data")
        시도명 = parts[idx + 1]
        학교급명 = parts[idx + 2]
    except Exception:
        raise ValueError(f"❌ input_path 구조 오류: {input_path}")

    # 파일명으로 국공립/사립 구분
    base_filename = os.path.basename(input_path)
    if "국공립" in base_filename:
        school_type = "국공립"
    elif "사립" in base_filename:
        school_type = "사립"
    else:
        school_type = "Unknown"

    # 저장할 기본 경로
    base_output_dir = os.path.join(output_dir, 시도명, 학교급명, school_type)
    os.makedirs(base_output_dir, exist_ok=True)

    safe_school_name = school_name.replace(" ", "_")

    for division, table_html in tables.items():
        df = pd.read_html(table_html)[0]

        save_filename = f"{file_year}_{school_type}_{safe_school_name}_{division}.csv"
        save_path = os.path.join(base_output_dir, save_filename)

        df.to_csv(save_path, index=False, encoding="utf-8-sig")
        print(f"✅ 저장 완료: {save_path}")

def process_all_files(input_dir: str, output_dir: str):
    for root, _, files in os.walk(input_dir):
        if not root.endswith(("2022", "2023", "2024")):
            continue  # 년도 폴더만 대상으로

        folder_year_str = os.path.basename(root)
        if not folder_year_str.isdigit():
            continue

        folder_year = int(folder_year_str)

        for file in files:
            if not (file.endswith(".xls") or file.endswith(".xlsx")):
                continue

            input_path = os.path.join(root, file)

            parsed = parse_html_file(input_path, folder_year)
            if parsed:
                save_parsed_tables(parsed, input_path, output_dir)

In [18]:
# input_dir = "data"
# output_dir = "Database/sample_schoolinfo"

# process_all_files(input_dir, output_dir)

## 평균 작업

In [24]:
import os
import glob
import pandas as pd
import re

def make_average_table_all(input_base_dir: str, output_dir: str):
    """
    전체 폴더를 돌면서 년도+테이블 종류별 평균 테이블 생성
    """
    os.makedirs(output_dir, exist_ok=True)

    years = [2022, 2023, 2024]
    kinds = ["세입예산", "세출예산", "세입결산", "세출결산"]

    for year in years:
        for kind in kinds:
            file_list = glob.glob(os.path.join(input_base_dir, "**", f"{year}_*_{kind}.csv"), recursive=True)
            
            if not file_list:
                print(f"⚠️ {year} {kind} 파일 없음, 스킵")
                continue

            print(f"✅ {year} {kind} 파일 {len(file_list)}개 찾음")

            amount_frames = []
            text_frame = None

            for file_path in file_list:
                try:
                    df = pd.read_csv(file_path)
                except Exception as e:
                    print(f"❌ 파일 읽기 실패: {file_path} / {e}")
                    continue

                if "금액" not in df.columns:
                    print(f"⚠️ '금액' 열 없음: {file_path}")
                    continue

                if text_frame is None:
                    text_frame = df.drop(columns=["금액"]).copy()

                amount_series = pd.to_numeric(df["금액"], errors="coerce")
                amount_frames.append(amount_series)

            if not amount_frames:
                print(f"❌ 금액 데이터 없음: {year} {kind}")
                continue

            # 금액 평균 계산
            amount_df = pd.concat(amount_frames, axis=1)
            amount_mean = amount_df.mean(axis=1)

            # 결과 테이블 만들기
            result_df = text_frame.copy()
            result_df["평균금액"] = amount_mean

            # 저장
            save_path = os.path.join(output_dir, f"{year}_{kind}_평균.csv")
            result_df.to_csv(save_path, index=False, encoding="utf-8-sig")
            print(f"✅ 저장 완료: {save_path}")

In [26]:
make_average_table_all(
    input_base_dir="Database/sample_schoolinfo",
    output_dir="Database/sample_schoolinfo/result"
)

✅ 2022 세입예산 파일 63개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/2022_세입예산_평균.csv
✅ 2022 세출예산 파일 63개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/2022_세출예산_평균.csv
✅ 2022 세입결산 파일 63개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/2022_세입결산_평균.csv
✅ 2022 세출결산 파일 63개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/2022_세출결산_평균.csv
✅ 2023 세입예산 파일 63개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/2023_세입예산_평균.csv
✅ 2023 세출예산 파일 63개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/2023_세출예산_평균.csv
✅ 2023 세입결산 파일 63개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/2023_세입결산_평균.csv
✅ 2023 세출결산 파일 63개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/2023_세출결산_평균.csv
✅ 2024 세입예산 파일 63개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/2024_세입예산_평균.csv
✅ 2024 세출예산 파일 63개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/2024_세출예산_평균.csv
✅ 2024 세입결산 파일 63개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/2024_세입결산_평균.csv
✅ 2024 세출결산 파일 63개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/2024_세출결산_평균.csv


### 지역별 합계

In [1]:
import os
import glob
import pandas as pd

def make_average_table_all_with_region_in_filename(input_base_dir: str, output_dir: str):
    """
    전체 폴더를 돌면서 '연도 + 지역 + 테이블 종류'별 평균 테이블 생성 (파일명에 지역명 포함)
    """
    os.makedirs(output_dir, exist_ok=True)

    years = [2022, 2023, 2024]
    kinds = ["세입예산", "세출예산", "세입결산", "세출결산"]

    for year in years:
        for kind in kinds:
            # 대상 파일 모두 찾기
            file_list = glob.glob(os.path.join(input_base_dir, "**", f"{year}_*_{kind}.csv"), recursive=True)
            file_list = [f for f in file_list if "result" not in f.split(os.sep)]  # result 폴더 제외

            if not file_list:
                print(f"⚠️ {year} {kind} 파일 없음, 스킵")
                continue

            # 지역별 파일 분류
            region_files = {}
            for file_path in file_list:
                parts = file_path.split(os.sep)
                try:
                    region_idx = parts.index("sample_schoolinfo") + 1
                    region = parts[region_idx]
                except ValueError:
                    print(f"❌ region 추출 실패: {file_path}")
                    continue

                if region not in region_files:
                    region_files[region] = []
                region_files[region].append(file_path)

            for region, region_file_list in region_files.items():
                print(f"✅ {year} {kind} {region} 파일 {len(region_file_list)}개 찾음")

                amount_frames = []
                text_frame = None

                for file_path in region_file_list:
                    try:
                        df = pd.read_csv(file_path)
                    except Exception as e:
                        print(f"❌ 파일 읽기 실패: {file_path} / {e}")
                        continue

                    if "금액" not in df.columns:
                        print(f"⚠️ '금액' 열 없음: {file_path}")
                        continue

                    if text_frame is None:
                        text_frame = df.drop(columns=["금액"]).copy()

                    amount_series = pd.to_numeric(df["금액"], errors="coerce")
                    amount_frames.append(amount_series)

                if not amount_frames:
                    print(f"❌ 금액 데이터 없음: {year} {kind} {region}")
                    continue

                # 평균 계산
                amount_df = pd.concat(amount_frames, axis=1)
                amount_mean = amount_df.mean(axis=1)

                # 결과 테이블
                result_df = text_frame.copy()
                result_df["평균금액"] = amount_mean

                # 저장 경로
                filename = f"{year}_{region}_{kind}_평균.csv"
                save_path = os.path.join(output_dir, filename)
                result_df.to_csv(save_path, index=False, encoding="utf-8-sig")
                print(f"✅ 저장 완료: {save_path}")

In [5]:
make_average_table_all_with_region_in_filename(
    input_base_dir="Database/sample_schoolinfo",
    output_dir="Database/sample_schoolinfo/result/average"
)

✅ 2022 세입예산 광주 파일 1개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/average/2022_광주_세입예산_평균.csv
✅ 2022 세입예산 제주 파일 1개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/average/2022_제주_세입예산_평균.csv
✅ 2022 세입예산 경북 파일 4개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/average/2022_경북_세입예산_평균.csv
✅ 2022 세입예산 경기 파일 16개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/average/2022_경기_세입예산_평균.csv
✅ 2022 세입예산 인천 파일 3개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/average/2022_인천_세입예산_평균.csv
✅ 2022 세입예산 충북 파일 4개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/average/2022_충북_세입예산_평균.csv
✅ 2022 세입예산 전북 파일 4개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/average/2022_전북_세입예산_평균.csv
✅ 2022 세입예산 전남 파일 4개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/average/2022_전남_세입예산_평균.csv
✅ 2022 세입예산 대구 파일 1개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/average/2022_대구_세입예산_평균.csv
✅ 2022 세입예산 대전 파일 1개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/average/2022_대전_세입예산_평균.csv
✅ 2022 세입예산 서울 파일 8개 찾음
✅ 저장 완료: Database/sample_

### 지역별 합계

In [3]:
import os
import glob
import pandas as pd

def make_sum_table_all_with_region_in_filename(input_base_dir: str, output_dir: str):
    """
    전체 폴더를 돌면서 '연도 + 지역 + 테이블 종류'별 합산 테이블 생성 (파일명에 지역명 포함)
    """
    os.makedirs(output_dir, exist_ok=True)

    years = [2022, 2023, 2024]
    kinds = ["세입예산", "세출예산", "세입결산", "세출결산"]

    for year in years:
        for kind in kinds:
            # 대상 파일 모두 찾기
            file_list = glob.glob(os.path.join(input_base_dir, "**", f"{year}_*_{kind}.csv"), recursive=True)
            file_list = [f for f in file_list if "result" not in f.split(os.sep)]  # result 폴더 제외

            if not file_list:
                print(f"⚠️ {year} {kind} 파일 없음, 스킵")
                continue

            # 지역별 파일 분류
            region_files = {}
            for file_path in file_list:
                parts = file_path.split(os.sep)
                try:
                    region_idx = parts.index("sample_schoolinfo") + 1
                    region = parts[region_idx]
                except ValueError:
                    print(f"❌ region 추출 실패: {file_path}")
                    continue

                if region not in region_files:
                    region_files[region] = []
                region_files[region].append(file_path)

            for region, region_file_list in region_files.items():
                print(f"✅ {year} {kind} {region} 파일 {len(region_file_list)}개 찾음")

                amount_frames = []
                text_frame = None

                for file_path in region_file_list:
                    try:
                        df = pd.read_csv(file_path)
                    except Exception as e:
                        print(f"❌ 파일 읽기 실패: {file_path} / {e}")
                        continue

                    if "금액" not in df.columns:
                        print(f"⚠️ '금액' 열 없음: {file_path}")
                        continue

                    if text_frame is None:
                        text_frame = df.drop(columns=["금액"]).copy()

                    amount_series = pd.to_numeric(df["금액"], errors="coerce")
                    amount_frames.append(amount_series)

                if not amount_frames:
                    print(f"❌ 금액 데이터 없음: {year} {kind} {region}")
                    continue

                # 합계 계산
                amount_df = pd.concat(amount_frames, axis=1)
                amount_sum = amount_df.sum(axis=1)

                # 결과 테이블
                result_df = text_frame.copy()
                result_df["합계금액"] = amount_sum

                # 저장 경로
                filename = f"{year}_{region}_{kind}_합계.csv"
                save_path = os.path.join(output_dir, filename)
                result_df.to_csv(save_path, index=False, encoding="utf-8-sig")
                print(f"✅ 저장 완료: {save_path}")

In [6]:
make_sum_table_all_with_region_in_filename(
    input_base_dir="Database/sample_schoolinfo",
    output_dir="Database/sample_schoolinfo/result/sum"
)

✅ 2022 세입예산 광주 파일 1개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/sum/2022_광주_세입예산_합계.csv
✅ 2022 세입예산 제주 파일 1개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/sum/2022_제주_세입예산_합계.csv
✅ 2022 세입예산 경북 파일 4개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/sum/2022_경북_세입예산_합계.csv
✅ 2022 세입예산 경기 파일 16개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/sum/2022_경기_세입예산_합계.csv
✅ 2022 세입예산 인천 파일 3개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/sum/2022_인천_세입예산_합계.csv
✅ 2022 세입예산 충북 파일 4개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/sum/2022_충북_세입예산_합계.csv
✅ 2022 세입예산 전북 파일 4개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/sum/2022_전북_세입예산_합계.csv
✅ 2022 세입예산 전남 파일 4개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/sum/2022_전남_세입예산_합계.csv
✅ 2022 세입예산 대구 파일 1개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/sum/2022_대구_세입예산_합계.csv
✅ 2022 세입예산 대전 파일 1개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/sum/2022_대전_세입예산_합계.csv
✅ 2022 세입예산 서울 파일 8개 찾음
✅ 저장 완료: Database/sample_schoolinfo/result/sum/2022_서울_세입예산_합계.cs

### 지역별 합계 파일 합치기

In [8]:
import os
import pandas as pd
import glob

def merge_sum_tables_by_kind_and_year(input_dir: str, output_dir: str):
    """
    지역별로 나뉜 합계 파일들을 테이블 종류+연도별로 통합하고, 지역명을 컬럼으로 추가
    """
    os.makedirs(output_dir, exist_ok=True)

    kinds = ["세입예산", "세출예산", "세입결산", "세출결산"]
    years = [2022, 2023, 2024]

    for year in years:
        for kind in kinds:
            all_dfs = []

            # 파일 이름 패턴: 2022_경남_세입결산_합계.csv
            file_list = glob.glob(os.path.join(input_dir, f"{year}_*_{kind}_합계.csv"))
            if not file_list:
                print(f"⚠️ {year} {kind} 파일 없음, 스킵")
                continue

            for file_path in file_list:
                df = pd.read_csv(file_path)

                # 파일명에서 지역명 추출
                basename = os.path.basename(file_path)
                parts = basename.split("_")
                if len(parts) < 4:
                    print(f"❌ 파일명 이상: {basename}")
                    continue

                region = parts[1]  # 2022_경남_세입결산_합계.csv → parts[1] = 지역명
                df["지역명"] = region

                all_dfs.append(df)

            if not all_dfs:
                print(f"⚠️ {year} {kind} 데이터 없음, 스킵")
                continue

            # 합치기
            merged_df = pd.concat(all_dfs, ignore_index=True)

            # 저장
            save_path = os.path.join(output_dir, f"지역별_{year}_{kind}_전체.csv")
            merged_df.to_csv(save_path, index=False, encoding="utf-8-sig")
            print(f"✅ 저장 완료: {save_path}")

In [9]:
merge_sum_tables_by_kind_and_year(
    input_dir="Database/sample_schoolinfo/result/sum",
    output_dir="Database/sample_schoolinfo/result/merged"
)

✅ 저장 완료: Database/sample_schoolinfo/result/merged/지역별_2022_세입예산_전체.csv
✅ 저장 완료: Database/sample_schoolinfo/result/merged/지역별_2022_세출예산_전체.csv
✅ 저장 완료: Database/sample_schoolinfo/result/merged/지역별_2022_세입결산_전체.csv
✅ 저장 완료: Database/sample_schoolinfo/result/merged/지역별_2022_세출결산_전체.csv
✅ 저장 완료: Database/sample_schoolinfo/result/merged/지역별_2023_세입예산_전체.csv
✅ 저장 완료: Database/sample_schoolinfo/result/merged/지역별_2023_세출예산_전체.csv
✅ 저장 완료: Database/sample_schoolinfo/result/merged/지역별_2023_세입결산_전체.csv
✅ 저장 완료: Database/sample_schoolinfo/result/merged/지역별_2023_세출결산_전체.csv
✅ 저장 완료: Database/sample_schoolinfo/result/merged/지역별_2024_세입예산_전체.csv
✅ 저장 완료: Database/sample_schoolinfo/result/merged/지역별_2024_세출예산_전체.csv
✅ 저장 완료: Database/sample_schoolinfo/result/merged/지역별_2024_세입결산_전체.csv
✅ 저장 완료: Database/sample_schoolinfo/result/merged/지역별_2024_세출결산_전체.csv


#### 확인

In [27]:
df = pd.read_csv("Database/sample_schoolinfo/result/2022_세입결산_평균.csv")
df


,과목,과목.1,과목.2,과목.3,평균금액
0,장,관,항,목,NaN
1,이전수입,이전수입,이전수입,이전수입,1.633259e+09
2,NaN,중앙정부이전수입,중앙정부이전수입,중앙정부이전수입,1.750000e+06
3,NaN,NaN,국고보조금,국고보조금,1.750000e+06
4,NaN,NaN,NaN,국고보조금,1.750000e+06
...,...,...,...,...,...
58,NaN,NaN,정산대상재원사용잔액,정산대상재원사용잔액,2.644607e+06
59,NaN,NaN,NaN,정산대상재원사용잔액,5.956330e+06
60,NaN,NaN,이월금,이월금,3.045653e+07
61,NaN,NaN,NaN,이월사업비,2.924700e+07


### 삭제 코드 주의!!!

In [43]:
# def delete_year_folders(base_dir="data"):
#     for region in os.listdir(base_dir):
#         region_path = os.path.join(base_dir, region)
#         if not os.path.isdir(region_path):
#             continue
#         for school_level in ["초등", "중등", "고등"]:
#             level_path = os.path.join(region_path, school_level)
#             if not os.path.isdir(level_path):
#                 continue

#             for year in ["2022", "2023", "2024"]:
#                 year_path = os.path.join(level_path, year)
#                 if os.path.isdir(year_path):
#                     print(f"[DELETE] 폴더 삭제: {year_path}")
#                     shutil.rmtree(year_path)

# delete_year_folders()